In [1]:
# -*- coding: utf-8 -*-
"""
@author: aeidemlan
"""
import datetime
import pandas as pd

from read import read_bspp_table, read_configuration
from tools import translate_id_into_label

In [4]:
nrows = 1000000000000000

In [5]:
tab = read_bspp_table("GestionMMA_HistoriqueMMAStatutOperationnel",
                      skiprows=0, nrows = nrows,
                      usecols=[1,2,4,5])
# 'IdMMASelection' joue le r�le de num�ro d'intervention par
# v�hicule

tab['date'] = pd.to_datetime(tab['DateStatutOperationnelMMA'],
                      format='%Y-%m-%d %H:%M:%S')
del tab['DateStatutOperationnelMMA']
tab['date'].value_counts() #beaucoup de date � la millisecondes prises
# plusieurs fois.
# C'est moins le cas quand on fait tab[test > '2013']
# tab[test > '2013'].DateStatutOperationnelMMA.value_counts()
#=> on peut imaginer qu'il y a eu une correction avant 2013
print(tab.date.dt.year.value_counts())
#2015    4452425
#2014    4238433
#2013    4081145
#2012    3974965
#2011    3675311
#2016    2701680
#2010      77559


def voir_intervention(num_intervention, from_table=tab):
    ''' pour facilement voir une intervention '''
    tab = from_table[from_table['IdMMASelection'] == num_intervention]
    return tab.sort_values('date')

2015    4452425
2014    4238433
2013    4081145
2012    3974965
2011    3675311
2016    2701680
2010      77559
Name: date, dtype: int64


C:\git\BSPP1\adagio\read.py:43: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.
  usecols=usecols,


In [6]:
statut_op = read_configuration('StatutOperationnel')
tab = translate_id_into_label('StatutOperationnel',
                              tab, statut_op, method='merge')
#
# sit on veut d'autres infos de StatutOperationnel
#tab = tab.merge(statut_op[['IdStatutOperationnel', 'LibelleStatutOperationnel', 'Disponibilite']])
#del tab['IdStatutOperationnel']
tab.rename(columns={
    'StatutOperationnel':'statut',
    }, inplace=True)
#tab['Disponibilite'].astype(bool)

print(voir_intervention('884767'))

var_utiles =  ['IdMMA','statut','date', 'IdMMASelection']
utiles = tab[tab['IdMMASelection'] > '0']
# TODO: comprendre ce que c'est quand pas d'intervention

# on a des doublons
len(utiles) == len(utiles.drop_duplicates(['IdMMA', 'IdMMASelection', 'statut']))
# => on fait les bourrins : on retire les interventions concern�es
# TODO: faire mieux
avec_doublons = utiles[utiles.duplicated(['IdMMA', 'IdMMASelection', 'statut'])]['IdMMASelection']
utiles = utiles[~utiles['IdMMASelection'].isin(avec_doublons.unique())]

bon_format = utiles[var_utiles].set_index(['IdMMA', 'IdMMASelection', 'statut']).unstack()
bon_format.columns = bon_format.columns.levels[1]
bon_format.isnull().sum()

        IdMMA IdStatutOperationnel IdMMASelection                    date
5001170   449                  102         884767 2012-04-25 11:33:36.547
5001182   449                    1         884767 2012-04-25 11:34:37.297
5001188   449                    7         884767 2012-04-25 11:35:08.533
5001190   449                  102         884767 2012-04-25 11:35:10.203
5001193   449                    1         884767 2012-04-25 11:35:14.390
5001194   449                    2         884767 2012-04-25 11:35:18.280
5001195   449                    7         884767 2012-04-25 11:35:20.413
5001196   449                    9         884767 2012-04-25 11:35:24.857


C:\git\BSPP1\adagio\read.py:43: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.
  usecols=usecols,


statut
AR sur les lieux                                                     3295148
Accueil                                                              3295185
Acquittement Message Information                                     3295168
Annulation sélection                                                 3295186
Arrivée hôpital                                                      2906963
CRS sur les lieux                                                    3295186
DDE sur les lieux                                                    3295184
DSM sur les lieux                                                    3295181
Demande SMUR                                                         3295174
Demande confirmation d'appel et d'adresse                            3288151
Demande police ou gendarmerie                                        3294950
Disponible                                                            411741
EDF sur les lieux                                                    

In [11]:
print(len(avec_doublons.unique()))
print(len(utiles))
print('on a en tout ', len(bon_format), ' inteventions')
# 3 295 188

267841
15715628
on a en tout  3295188  inteventions


In [12]:
## pr�pare un mod�le de regression
statut_d_une_intervention_classique = [
    'Instance de sélection',
    'Sélection',
    'Instance départ',
    'Parti',
    'Sur les lieux ',
    'Transport hôpital',
    'Arrivée hôpital',
    'Quitte hôpital',
    'Indisponible Montée en GARDE',
    'Indisponible en Transit',
    # disponible
    'Rentré',
    'Disponible',
    ]
# tous les autres c'est indisponible


### on cr�e maintenant les variables utiles pour la r�gression.
# on selectionnce uniquement quand on a rempli la liste suivante
var_hopital = ['Transport hôpital','Arrivée hôpital','Quitte hôpital']
bon_format['hopital'] = bon_format[var_hopital].notnull().sum(1) > 0


statut_obligatoire = [
    'Instance départ',
    'Parti',
    'Sur les lieux',
    # disponible
    'Rentré',
    'Disponible',
    ]
cond_tout_rempli = bon_format[statut_obligatoire].isnull().sum(1) == 0
tout_rempli = bon_format.loc[cond_tout_rempli, statut_obligatoire + ['hopital']]
# => on retire un tiers des intervention

tout_rempli.reset_index(inplace=True)

#####  les infos sur le MMA ######
mma = read_configuration('MMA')
# on utilise pas l'originel
del mma['IdFamilleMMAOriginelle']
# TODO: retirer d'autres variables
#TODO savoir ce qu'est un Omnibus
to_remove = ['ImmatriculationBSPPMMA', 'ImmatriculationAdministrativeMMA',
             'RFGI', 'GSM', 'Actif', 'Strada', 'Omnibus',
             'Associe', 'OrdreGTA', 'Disponible', 'Observation',
             'IdStatutOperationnel']
mma.drop(to_remove, axis=1, inplace=True)

# famille mma
famille_mma = read_configuration('FamilleMMA')
del famille_mma['FamilleMMA'] # qui est vide
mma.rename(columns={'IdFamilleMMAOperationnelle': 'IdFamilleMMA'}, inplace=True)
mma = translate_id_into_label('FamilleMMA',
                              mma, famille_mma, method='merge')


var_nombre = [var for var in famille_mma.columns if 'Nombre' in var]
famille_mma = famille_mma[['LibelleFamilleMMA'] + var_nombre]
mma = mma.merge(famille_mma, left_on='FamilleMMA', right_on='LibelleFamilleMMA')
del mma['LibelleFamilleMMA']

# Apparentance
appartenance = read_configuration('MoyenSecoursAppartenance')
mma = translate_id_into_label('MoyenSecoursAppartenance',
                              mma, appartenance, method='merge')

#IdFamilleMMAModele
assert all(mma.IdFamilleMMAModele == '1')
# donc variable ininteressante
# donc MMAModele = read_configuration('FamilleMMAModele') ne sert � rien
# donc:
del mma['IdFamilleMMAModele']
# TODO: regarder les autres variables ident
# IdLieuStationnementOperationnel
# et
# IdAffectationAdministrative
tout_rempli = tout_rempli.merge(mma, on='IdMMA')



#####  les infos sur les interventions ######
# tables concern�e
#        "Appel112_MMASelection",
#        "Appel112_MMARessourcePartageeSelection",
#        "GestionMMA_FamilleMMASelection"
selection = read_bspp_table("Appel112_MMASelection", nrows=nrows)


type_selection = read_bspp_table("Appel112_R_TypeSelection", nrows=nrows)
#   IdTypeSelection AbregeTypeSelection          LibeleTypeSelection
#0                0                   A                       Annul�
#1                1                   D                   D�finitive
#etc...
selection = selection.merge(type_selection)
del selection['IdTypeSelection']

resume = read_bspp_table("Appel112_InterventionResume",
                         usecols=[0,1,2,3,4, 9], nrows=nrows)
selection = selection.merge(resume, on = 'IdIntervention')
# TODO: on a d'autre chose à faire, il faut faire le merge avec
# IdMMA par exemple. En attendant
del selection['IdMMA']
#del selection['IdIntervention']


tout_rempli = tout_rempli.merge(selection, on = 'IdMMASelection',
                                how='left')

tout_rempli.rename(columns={
    'Disponible_x':'Disponible',
    }, inplace=True)

var_label = ['ObservationsPourMMA', 'LibelleMotif', 'LibeleTypeSelection', 'ImmatriculationBSPPMMA',
'LibelleMoyenSecoursAppartenance', 'LibelleFamilleMMA']






C:\git\BSPP1\adagio\read.py:43: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.
  usecols=usecols,


In [13]:
print(len(tout_rempli))
print(1 - len(tout_rempli)/len(bon_format))

2151572
0.3470563743252282


In [21]:
tout_rempli['Affectation_du_vehicule'] = tout_rempli.Libelle_GTA.str.split("_").str[2]

In [22]:
sum(tout_rempli['Cstc'] == tout_rempli['Affectation_du_vehicule']) / len(tout_rempli['Cstc'])

0.44488820267227869

In [18]:
sum(tout_rempli['Cstc'] == tout_rempli['Affectation_du_vehicule']) / len(tout_rempli['Cstc'])

0.44488820267227869

In [30]:
tout_rempli.to_csv('tout_rempli.csv', index=False)

In [28]:
df = tout_rempli
df.values.nbytes + df.index.nbytes + df.columns.nbytes

602440432

In [29]:
602440432/1e6

602.440432

In [17]:
print(tout_rempli.Cstc.unique())
len(tout_rempli.Cstc.unique())

['MTMA' 'BLAN' 'ROUS' 'STHO' 'CHPT' 'BSLT' 'CHTO' 'MALA' 'LAND' 'STOU'
 'CCL1' 'PIER' 'STDE' 'LACO' 'DAUP' 'PLAI' 'CHAR' 'MENI' 'COBI' 'POIS'
 'PROY' 'GREN' 'VINC' 'MTRL' 'NATI' 'BOND' 'VIMB' 'DRAN' 'PANT' 'NOGT'
 'CHLY' 'PARM' 'MALF' 'MASS' 'RUNG' 'CHOI' 'CHPY' 'STMR' 'BOUL' 'ISSY'
 'MTGE' 'BGLR' 'CLAM' 'GENN' 'LEVA' 'AUTE' 'SEVR' 'AULN' 'ASNI' 'PLCL'
 'COBE' 'ANTO' 'BLME' 'CBVE' 'AUBE' 'BITC' 'CLIC' 'PCDG' 'CCL2' 'SUCY'
 'NOIS' 'CRET' 'NEUI' 'VILC' 'VISG' 'VIJF' 'UESR' 'MEUD' 'SEVI' 'NANT'
 'NBCG' 'STCL' 'LIVR' 'PUTX' 'IVRY' 'TREM' 'VITR' 'RUEI' 'CCL3' nan 'STEC'
 None 'CDS1' 'POUC' 'ORLY' 'CASG' 'CFC' 'HEGP' 'CSM' 'CCHY' '16' 'NBCL'
 'CASS' 'CIR' 'JOIN' 'NBCR' 'CASJ' 'LGS' 'MONN' 'visg' 'CASC' 'NBCV' 'SDIS'
 'NBCP' 'CDS2' '27' '26' '13' 'GPOR' 'CMAI' '23' 'CSC' 'SIFR']


113

In [ ]:
print(tout_rempli.Affectation_du_vehicule.unique())
len(tout_rempli.Affectation_du_vehicule.unique())

In [ ]:
print(list(set(tout_rempli.Cstc.unique()).intersection(tout_rempli.Affectation_du_vehicule.unique())))
len(set(tout_rempli.Cstc.unique()).intersection(tout_rempli.Affectation_du_vehicule.unique()))

In [ ]:
tout_rempli[['Cstc', 'Affectation_du_vehicule']]